In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Initilizing valid kerboros token to interact with cloudera 
# Run this cell when kerberos auth is expired

!echo "User :$USER"
yourPwd = input("Enter your password :")

!kdestroy
!echo "$yourPwd" | kinit
!klist

In [ ]:
import findspark

# Initilize find spark with valid spark home
findspark.init('/opt/cloudera/parcels/CDH-6.1.0-1.cdh6.1.0.p0.770702/lib/spark', edit_rc=True)

import pyspark
import pandas as pd
import numpy as np

In [ ]:
from pyspark.sql.functions import *

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option("display.max_rows", None)
pd.set_option("expand_frame_repr",True)
pd.set_option('max_colwidth',100)

In [ ]:
import pandas as pd 
import numpy as np 
from datetime import date, timedelta, datetime 
import time 
import pyspark # only run this after findspark.init() 
from pyspark.sql import SparkSession, SQLContext 
from pyspark.context import SparkContext 
from pyspark.sql.functions import * 
from pyspark.sql.types import * 
import pyspark 
from pyspark.sql import SparkSession 

In [ ]:
spark = SparkSession. \
    builder. \
    enableHiveSupport(). \
    appName('standard placeiq queries'). \
    master('yarn'). \
    getOrCreate()

from pyspark.sql.functions import *

In [ ]:
cat_id = spark.sql("SELECT * FROM placeiq_analytics.category_id_count_per_chain_new")

In [ ]:
cat_id.show(20)

In [ ]:
cat_id_desc = spark.sql("SELECT * FROM placeiq_analytics.category_id_count_per_chain_new WHERE chain_id = 'lowes-home-improvement' ORDER BY dt DESC")

In [ ]:
cat_id_desc.show(10)

In [ ]:
#top_20_place_rows = spark.sql("select * FROM placeiq_db_new.place LIMIT 20")

In [ ]:
#top_20_place_rows

In [ ]:
#top_20_place_rows.show(20)

In [ ]:
#top_20_visit_rows = spark.sql("select * FROM placeiq_db_new.visit LIMIT 20")

In [ ]:
#top_20_visit_rows

In [ ]:
# run_name = config_dict['run_name'][0]
# chain_ids = config_dict['chain_ids']
# descriptions = config_dict['descriptions']
# date_range = config_dict['date_range']
# timeslice = config_dict['timeslice'][0]
# dow_start = config_dict['dow_start']
# states = config_dict['states']
# dmas = config_dict['dmas']
# chain_desc_operator = config_dict['chain_desc_operator'][0]
# state_dma_operator = config_dict['state_dma_operator'][0]
# usage_case = config_dict['usage_case'][0]

run_name = 'lowes_visits'
chain_ids = ['lowes-home-improvement']
descriptions = ''
date_range = ['2021-01-24','2022-07-16'] #example['2022-03-20','2022-03-26']
timeslice = 'weekly'
dow_start = 'Sun'
states = ''
dmas = ''
chain_desc_operator = 'AND'
state_dma_operator = 'AND'
usage_case = 'General'

In [ ]:
if(len(dow_start) > 0):
    dow_start = dow_start[0]


# Step 2 - Use the description and the chain_ids to filter from placeiq_analytics.category_id_count_per_chain

cat_id_table = spark.sql("SELECT * FROM placeiq_analytics.category_id_count_per_chain_new")

# if len(chain_ids) > 0:
#     cat_id_table = cat_id_table.filter(col('chain_id').isin(chain_ids))

# # assuming an "AND" between chain_id and description
# if len(descriptions) > 0:
#     cat_id_table = cat_id_table.filter(col('description').isin(descriptions))

######################################################################################

In [ ]:
if ((len(chain_ids) > 0) & (len(descriptions) > 0)):
    if chain_desc_operator == 'AND':
        cat_id_table = cat_id_table.filter((col('chain_id').isin(chain_ids)) & (col('description').isin(descriptions)))
    else:
        cat_id_table = cat_id_table.filter((col('chain_id').isin(chain_ids)) | (col('description').isin(descriptions)))
elif (len(chain_ids)>0):
    cat_id_table = cat_id_table.filter(col('chain_id').isin(chain_ids))
elif (len(descriptions) > 0):
    cat_id_table = cat_id_table.filter(col('description').isin(descriptions))
else:
    cat_id_table = cat_id_table

In [ ]:
# drop the count column
cat_id_table = cat_id_table.select('chain_id','category_id','description')

# drop all non-chains
cat_id_table = cat_id_table.filter(~(col('chain_id') == 'non_chain'))

# Step 3 - Explode place table on category_ids and join on chain_id and category_id from Step 2

# read in place table
place_df = spark.sql("SELECT * FROM placeiq_db_new.place")

place_exploded = place_df.withColumn('split',split(col('category_ids'), ','))\
                   .withColumn('exploded', explode(col('split')))\
                   .drop('split').withColumnRenamed('exploded','category_id')

#place_filtered = place_exploded.join(cat_id_table, ['chain_id','category_id'])
place_filtered = place_exploded.join(cat_id_table, ['chain_id','category_id'])

place_filtered = place_filtered.drop('dt')

In [ ]:
"""
## Step 4 - Apply appropriate filters, remove duplicates

filter for dma/state

For Analytics Use Case (narrow)
1. is_high_confidence='true'
2. Is_open = 'true'
3. Is_multiuse='false'
4. movement_source_key = '231'

Using Daily_Visit_Count (middle):
1. Just join a filtered daily_visit_count (using local_date) to filtered places table 
on spatial_id instead of starting from visit table.

For Measurement/Audience (broad):
1. Is_open='true'
2. Is_high_confidence='true'
"""

# read in visit table with dt between date_range[0] and date_range[1]
#date_range[0].replace("-","")



# filter place on dmas and state
# if len(dmas) > 0:
#      place_filtered = place_filtered.filter(col('dma_id').isin(dmas))

# # filter place on dmas and state - Assumes AND between dma and state
# if len(states) > 0:
#      place_filtered = place_filtered.filter(col('state').isin(states))

##########################################################################

In [ ]:
if ((len(dmas) > 0) & (len(states) > 0)):
    if state_dma_operator == 'AND':
        place_filtered = place_filtered.filter((col('dma_id').isin(dmas)) & (col('state').isin(states)))
    else:
        place_filtered = place_filtered.filter((col('dma_id').isin(dmas)) | (col('state').isin(states)))
elif (len(dmas)>0):
    place_filtered = place_filtered.filter(col('dma_id').isin(dmas))
elif (len(states) > 0):
    place_filtered = place_filtered.filter(col('state').isin(states))
else:
    place_filtered = place_filtered

##########################################################################

In [ ]:
#place_filtered.show(20)

In [ ]:
#location_count = place_filtered.groupBy('chain_id').agg(countDistinct('place_id').alias('num_locations')).sort('chain_id')

In [ ]:
if usage_case == 'Analytics':
    visit_df = spark.sql("""SELECT * FROM placeiq_db_new.visit WHERE dt
     BETWEEN '{}' and '{}'""".format(date_range[0].replace("-",""), date_range[1].replace("-","")))


    # ensure is_open = true, is_high_confidence = true, and is_multiuse = false
    place_filtered = place_filtered.filter(
                    (col('is_open') == 'true') & (col('is_high_confidence') == 'true') & (col('is_multiuse') == 'false')  
                    )

    # Step 5 - Join Filtered Visit Table to Filtered Place Table on spatial_id
    visit_df = visit_df.filter(col('movement_source_key')==231)

    place_visits = place_filtered.join(visit_df, ['spatial_id'])

    # Step 6 - Filter by comparing visit attribute score >= 0.5* ceiling score
    #          Remove duplicates based on device_key, spatial_id, dt

    pv_analytics = place_visits
    pv_analytics = pv_analytics.filter(
                    (col('visit_attribute_score') >= 0.5*(col('ceiling_score')))
                    )

    # select the important columns

    pv_analytics = pv_analytics.select('device_key','place_id','spatial_id','chain_id','category_id',
        'description','latitude','longitude','dma_id','unix_time','dt')

    # need to remove duplicates for the same (device_id, place_id, dt)
    pv_analytics = pv_analytics.dropDuplicates(['device_key','spatial_id','dt'])

    # Step 7 - aggregate at the daily level, leaving dma_id as one of the remaining columns
    daily_pv_analytics = pv_analytics.groupBy('chain_id','category_id','description','place_id','latitude','longitude','dma_id','dt') \
                                    .agg(count(lit(1)).alias('visit_count'))

    ## Step 8 - join to fact_normalization_by_dma_limited to create a multiplication factor column for
    #  each site visit (unless using daily_visit_count)

    # read in fact_normalization_by_dma_limited using dt = most recent dt (until we just have most recent date)

    dma_normal_factor = spark.sql("SELECT * from placeiq_db_new.fact_normalization_by_dma_limited")

    dma_normal_factor = dma_normal_factor.drop('dt')

    dma_normal_factor = dma_normal_factor.withColumn('norm_date', regexp_replace('norm_date','-',''))

    daily_normalized_analytics = daily_pv_analytics.join(dma_normal_factor, (dma_normal_factor.dma_code ==
                daily_pv_analytics.dma_id) & (dma_normal_factor.norm_date == daily_pv_analytics.dt))

    daily_normalized_analytics = daily_normalized_analytics.drop('dma_code','norm_date')

    ## add in norm_count to get to weighted visit_count

    daily_normalized_analytics = daily_normalized_analytics.withColumn("visits_norm", col('norm_factor') * col('visit_count')) \
                                                        .withColumn("dt", to_date("dt","yyyyMMdd"))

###########################################################################################################################
elif usage_case == 'General':
    place_filtered = place_filtered.filter(
                    (col('is_open') == 'true') & (col('is_multiuse') == 'false') #& (col('is_high_confidence') == 'true') 
                    )

    visit_df = spark.sql("""SELECT * FROM placeiq_db_new.daily_visit_counts WHERE dt
                            BETWEEN '{}' and '{}'""".format(date_range[0].replace("-",""), date_range[1].replace("-","")))

    daily_normalized_analytics = place_filtered.join(visit_df, ['spatial_id'])
    
    daily_normalized_analytics = daily_normalized_analytics.withColumn("dt", to_date("dt","yyyyMMdd"))

    # need to remove duplicates for the same (place_id, dt)
    daily_normalized_analytics = daily_normalized_analytics.drop_duplicates(['place_id','dt'])

elif usage_case =='Targeting':
    visit_df = spark.sql("""SELECT * FROM placeiq_db_new.visit WHERE dt
     BETWEEN '{}' and '{}'""".format(date_range[0].replace("-",""), date_range[1].replace("-","")))

    place_filtered = place_filtered.filter(
                    (col('is_open') == 'true') #& (col('is_high_confidence') == 'true') & (col('is_multiuse') == 'false')
                    )
    ## join to daily visits count
    place_visits = place_filtered.join(visit_df, ['spatial_id'])

    pv_analytics = place_visits
    pv_analytics = pv_analytics.filter(
                    (col('visit_attribute_score') >= 0.1*(col('ceiling_score')))
                    )

    pv_analytics = pv_analytics.select('device_key','place_id','spatial_id','chain_id','category_id',
        'description','latitude','longitude','dma_id','unix_time','dt')

    # need to remove duplicates for the same (device_id, place_id, dt)
    pv_analytics = pv_analytics.dropDuplicates(['device_key','spatial_id','dt'])
    
    # Step 7 - aggregate at the daily level, leaving dma_id as one of the remaining columns
    daily_normalized_analytics = pv_analytics.groupBy('chain_id','category_id','description','place_id','latitude','longitude','dma_id','dt') \
                                    .agg(count(lit(1)).alias('visits_norm')) 
                                    

    daily_normalized_analytics = daily_normalized_analytics.withColumn("dt", to_date("dt","yyyyMMdd"))

In [ ]:
#daily_normalized_analytics.show(20)

In [ ]:
# daily_normalized_analytics.groupBy('chain_id').agg(countDistinct('place_id').alias('num_locations')).sort('chain_id').show()
#daily_normalized_analytics.printSchema()

In [ ]:
# Step 9 - use time slice to group visit counts by appropriate aggregation level as provided by json

######### create dict for the different date formats

date_format_dict = {"daily":"yyyy-MM-dd",
                    "monthly":"yyyy-MM",
                    "yearly":"yyyy"}

# Spark considers Sunday to be the first day of the week and Saturday to be the last day of the week.
# You’ll need to pass in an optional lastDayOfWeek argument if you’d like to use a custom week definition.

In [ ]:
#########
if timeslice == 'weekly':
    timeslice_aggregated_df = daily_normalized_analytics.withColumn('week_end', next_day(col('dt'),dow_start))\
                                                     .withColumn('week_start', date_add(col('week_end'), -6)) \
                                                     .withColumn('timeslice', concat(col('week_start'), lit(' - '), col('week_end')))

    timeslice_aggregated_df = timeslice_aggregated_df.groupBy('chain_id','category_id','description','place_id','latitude','longitude','dma_id','timeslice') \
                                                     .agg(round(sum('visits_norm'),0).alias('visit_count'))
    
elif timeslice == 'quarterly':
    timeslice_aggregated_df = daily_normalized_analytics.withColumn('quarter', quarter(col('dt'))) \
                                                        .withColumn('year', date_format(col('dt'), 'yyyy')) \
                                                        .withColumn('timeslice', concat(col('year'), lit('-Q'), col('quarter')))
                                                        
    timeslice_aggregated_df = timeslice_aggregated_df.groupBy('chain_id','category_id','description','place_id','latitude','longitude','dma_id','timeslice') \
                                                     .agg(round(sum('visits_norm'),0).alias('visit_count'))

else:
    timeslice_aggregated_df = daily_normalized_analytics.groupBy('chain_id','category_id','description','place_id','latitude','longitude','dma_id',
        date_format("dt",date_format_dict[timeslice]). \
        alias('timeslice')).agg(round(sum('visits_norm'),0).alias('visit_count'))

In [ ]:
#timeslice_aggregated_df.show(20)

In [ ]:
# timeslice_aggregated_df.groupBy('chain_id').agg(countDistinct('place_id').alias('num_locations')).sort('chain_id').show()

In [ ]:
# Step 10 - Write to appropriate table. Decide on important columns. Be sure to include run name.


table_out = timeslice_aggregated_df

table_out = table_out.withColumn('run_name', lit(run_name))

table_out = table_out.withColumn('date_created', lit(date_format(current_timestamp(),"yyyy-MM-dd HH:mm")))

table_out = table_out.withColumn('usage_case', lit(usage_case))

#output_table.write.mode("overwrite").saveAsTable("placeiq_analytics.testing_hive_output")

In [ ]:
#table_out.show(20)

In [ ]:
#table_out.groupBy('chain_id').agg(countDistinct('place_id').alias('num_locations')).sort('chain_id').show()

if len(table_out.head(1)) == 0:
    print("no rows found")
    row_warning = [("no rows returned","check run config","check to make sure","there are stores","with" ,"If usage_case=Analytics is_open=true is_high_confidence=true and is_multiuse=false","if general check is_open=true is_multiuse=false","if targeting check is_open=true",0,run_name,'',usage_case)]
    col_names = ["chain_id","category_id","description","place_id","latitude","longitude","dma_id","timeslice","visit_count","run_name","date_created","usage_case"]
    table_out = spark.createDataFrame(data=row_warning,schema=col_names)

    table_out = table_out.withColumn('date_created',lit(date_format(current_timestamp(),"yyyy-MM-dd HH:mm")))
    table_out = table_out.select("chain_id","category_id","description","place_id","latitude","longitude","dma_id","timeslice","visit_count","run_name","date_created","usage_case")
    table_out.show()

In [ ]:
#table_out.show()

In [ ]:
df = table_out.toPandas()

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.place_id.nunique()

In [ ]:
df1 = df.copy()

In [ ]:
df.to_csv("aldiStores_testRun.csv", index=False)

In [ ]:
spark.stop()

In [ ]:
df.head()